**Except / Minus:**

- The **Except / Minus** operator returns **all rows** from **table_A** that **do not exist** in **table_B**.

- Returns rows from the result set of the **first SELECT** statement that are **not present** in the result set of the **second SELECT** statement.

   - **Query1 - Query2**
   - It also **eliminates duplicate** rows.

**What happens?**

- SQL Server first collects all **distinct rows** from the **first SELECT**.

- It then **removes** any rows that **exactly match a row in the second SELECT**.


**I have two tables and first table has 100 records and second table has 90 records. How to get those missed records?**

**Basic Rules:**

- The **number and the order** of the **columns** must be the **same** in **both the queries**.
- Corresponding columns must have **compatible data types**.
- It automatically **removes duplicates** (similar to DISTINCT).
  - If you want to **retain duplicates**, use **NOT EXISTS or LEFT JOIN ... IS NULL**.

**Valid in:**
- Oracle
- Snowflake
- Google BigQuery

**Not valid in:**
- **SQL Server:**
  - Use **EXCEPT** instead.
- **MySQL**:
  - Doesn't support **MINUS or EXCEPT**.
  - Use **LEFT JOIN ... IS NULL**.

- This is similar to **minus** operator in **oracle**.

**Syntax**

     SELECT <columns> FROM <TableA>
     [WHERE ...]
     EXCEPT
     SELECT <columns> FROM <TableB>
     [WHERE ...];

In [0]:
%sql
DROP TABLE IF EXISTS Except_TableA;

CREATE TABLE Except_TableA
(
 EmployeeID int,
 Name VARCHAR(50),
 Gender VARCHAR(10),
 Country VARCHAR(20),
 Designation VARCHAR(20)
);

INSERT INTO Except_TableA VALUES (101, 'Suhash', 'M', 'India', 'Manager');
INSERT INTO Except_TableA VALUES (102, 'Veena', 'F', 'India', 'Sr Manager');
INSERT INTO Except_TableA VALUES (103, 'Sathya', 'M', 'India', 'Associate');
INSERT INTO Except_TableA VALUES (104, 'Marc', 'M', 'US', 'Project Manager');
INSERT INTO Except_TableA VALUES (105, 'Brad', 'M', 'Sweden', 'Dy Manager');
INSERT INTO Except_TableA VALUES (106, 'Paul', 'M', 'AUS', 'Lead');
INSERT INTO Except_TableA VALUES (107, 'Srinidi', 'F', 'India', 'DGM');

SELECT * FROM Except_TableA;

EmployeeID,Name,Gender,Country,Designation
104,Marc,M,US,Project Manager
103,Sathya,M,India,Associate
105,Brad,M,Sweden,Dy Manager
102,Veena,F,India,Sr Manager
101,Suhash,M,India,Manager
107,Srinidi,F,India,DGM
106,Paul,M,AUS,Lead


In [0]:
%sql
DROP TABLE IF EXISTS Except_TableB;

CREATE TABLE Except_TableB
(
 EmployeeID int,
 Name VARCHAR(50),
 Gender VARCHAR(10),
 Country VARCHAR(20),
 Designation VARCHAR(20)
);

INSERT INTO Except_TableB VALUES (101, 'Swapna', 'F', 'India', 'DGM');
INSERT INTO Except_TableB VALUES (102, 'Tanvee', 'F', 'India', 'Manager');
INSERT INTO Except_TableB VALUES (103, 'Somesh', 'M', 'India', 'Jr Manager');
INSERT INTO Except_TableB VALUES (104, 'Marc', 'M', 'US', 'Project Manager'); -- common records b/n TableA and TableB
INSERT INTO Except_TableB VALUES (105, 'Brad', 'M', 'Sweden', 'Dy Manager');  -- common records b/n TableA and TableB
INSERT INTO Except_TableB VALUES (106, 'Lopa', 'F', 'UK', 'Lead');
INSERT INTO Except_TableB VALUES (107, 'Srinivas', 'M', 'India', 'AGM');

SELECT * FROM Except_TableB;

EmployeeID,Name,Gender,Country,Designation
103,Somesh,M,India,Jr Manager
104,Marc,M,US,Project Manager
105,Brad,M,Sweden,Dy Manager
102,Tanvee,F,India,Manager
107,Srinivas,M,India,AGM
101,Swapna,F,India,DGM
106,Lopa,F,UK,Lead


**1) Comparing Subsets of Columns**

In [0]:
%sql
SELECT EmployeeID
FROM Except_TableA

EXCEPT

SELECT EmployeeID
FROM Except_TableB;

EmployeeID


**2) Returns the unique rows from the left query that aren’t in the right query’s results**
- The **number and the order** of the **columns** are **same** in both the queries.
- The **data types** are **same**.

In [0]:
%sql
SELECT EmployeeID, Name, Gender, Country, Designation
FROM Except_TableA
EXCEPT
SELECT EmployeeID, Name, Gender, Country, Designation
FROM Except_TableB

EmployeeID,Name,Gender,Country,Designation
103,Sathya,M,India,Associate
102,Veena,F,India,Sr Manager
101,Suhash,M,India,Manager
107,Srinidi,F,India,DGM
106,Paul,M,AUS,Lead


**Order By:**
- If you want the **final result ordered**, you can put **ORDER BY** only after the **second SELECT**.

In [0]:
%sql
SELECT EmployeeID, Name, Gender, Country, Designation
FROM Except_TableA
EXCEPT
SELECT EmployeeID, Name, Gender, Country, Designation
FROM Except_TableB
ORDER BY EmployeeID;

EmployeeID,Name,Gender,Country,Designation
101,Suhash,M,India,Manager
102,Veena,F,India,Sr Manager
103,Sathya,M,India,Associate
106,Paul,M,AUS,Lead
107,Srinidi,F,India,DGM


**3) To retrieve all of the rows from Table B that does not exist in Table A, reverse the two queries as shown below**
- The **number and the order** of the **columns** are **same** in both the queries.
- The **data types** are **same**.

In [0]:
%sql
SELECT EmployeeID, Name, Gender, Country, Designation
FROM Except_TableB
EXCEPT
SELECT EmployeeID, Name, Gender, Country, Designation
FROM Except_TableA

EmployeeID,Name,Gender,Country,Designation
103,Somesh,M,India,Jr Manager
102,Tanvee,F,India,Manager
107,Srinivas,M,India,AGM
101,Swapna,F,India,DGM
106,Lopa,F,UK,Lead


**4) [NUM_COLUMNS_MISMATCH]**
- **Unequal number** of columns
- The **data types** are **same**.

In [0]:
%sql
SELECT EmployeeID, Name, Gender, Country, Designation
FROM Except_TableA
EXCEPT
SELECT EmployeeID, Name, Gender, Country
FROM Except_TableB

org.apache.spark.sql.catalyst.ExtendedAnalysisException: [NUM_COLUMNS_MISMATCH] EXCEPT can only be performed on inputs with the same number of columns, but the first input has 5 columns and the second input has 4 columns. SQLSTATE: 42826; line 1 pos 0;
'Except false
:- Project [EmployeeID#9451, Name#9452, Gender#9453, Country#9454, Designation#9455]
:  +- SubqueryAlias spark_catalog.default.Except_TableA
:     +- Relation spark_catalog.default.except_tablea[EmployeeID#9451,Name#9452,Gender#9453,Country#9454,Designation#9455] parquet
+- Project [EmployeeID#9456, Name#9457, Gender#9458, Country#9459]
   +- SubqueryAlias spark_catalog.default.Except_TableB
      +- Relation spark_catalog.default.except_tableb[EmployeeID#9456,Name#9457,Gender#9458,Country#9459,Designation#9460] parquet

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:55)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis0$33(CheckAnalysis.scala:643)

**4) [NUM_COLUMNS_MISMATCH]**
- **Number of Columns** are **Same**.
- Columns **order is different**.

In [0]:
%sql
SELECT Name, EmployeeID, Gender, Country, Designation
FROM Except_TableA
EXCEPT
SELECT EmployeeID, Name, Gender, Country, Designation
FROM Except_TableB

Name,EmployeeID,Gender,Country,Designation
Marc,104,M,US,Project Manager
Sathya,103,M,India,Associate
Brad,105,M,Sweden,Dy Manager
Veena,102,F,India,Sr Manager
Suhash,101,M,India,Manager
Srinidi,107,F,India,DGM
Paul,106,M,AUS,Lead


| TableA Field | TableB Field | Result     |
| ------------ | ------------ | ---------- |
| Name         | Id           | mismatched |
| Id           | Name         | mismatched |
| Gender       | Gender       | matched    |
| Country      | Country      | matched    |
| Designation  | Designation  | matched    |


- Because most rows have **different Name vs Id values**, **none matched.** So EXCEPT returned **all rows** from **TableA**.

**5) The column names do not have to be the same:**

- Even though **id ≠ emp_id** and **name ≠ emp_name**, this will still work as long as:

  - Both queries return the **same number of columns**.
  - The **data types** of corresponding columns match or are compatible.

**Summary**:

❌ **Column names:** Don't need to match.

✅ **Number of columns:** Must match.

✅ **Data types:** Must be compatible.

In [0]:
%sql
DROP TABLE IF EXISTS Except_TableC;

Create Table Except_TableC
(
 emp_id int,
 emp_name VARCHAR(50),
 Gender VARCHAR(10),
 Country VARCHAR(20),
 Designation VARCHAR(20)
);

INSERT INTO Except_TableC VALUES (101, 'Swapna', 'F', 'India', 'DGM');
INSERT INTO Except_TableC VALUES (102, 'Tanvee', 'F', 'India', 'Manager');
INSERT INTO Except_TableC VALUES (103, 'Somesh', 'M', 'India', 'Jr Manager');
INSERT INTO Except_TableC VALUES (104, 'Marc', 'M', 'US', 'Project Manager'); -- common records b/n TableA and TableB
INSERT INTO Except_TableC VALUES (105, 'Brad', 'M', 'Sweden', 'Dy Manager');  -- common records b/n TableA and TableB
INSERT INTO Except_TableC VALUES (106, 'Lopa', 'F', 'UK', 'Lead');
INSERT INTO Except_TableC VALUES (107, 'Srinivas', 'M', 'India', 'AGM');

SELECT * FROM Except_TableC;

emp_id,emp_name,Gender,Country,Designation
103,Somesh,M,India,Jr Manager
104,Marc,M,US,Project Manager
105,Brad,M,Sweden,Dy Manager
102,Tanvee,F,India,Manager
107,Srinivas,M,India,AGM
101,Swapna,F,India,DGM
106,Lopa,F,UK,Lead


In [0]:
%sql
SELECT EmployeeID, Name, Gender, Country, Designation
FROM Except_TableA
EXCEPT
SELECT emp_id, emp_name, Gender, Country, Designation
FROM Except_TableC;

EmployeeID,Name,Gender,Country,Designation
103,Sathya,M,India,Associate
102,Veena,F,India,Sr Manager
101,Suhash,M,India,Manager
107,Srinidi,F,India,DGM
106,Paul,M,AUS,Lead
